## Importing Dependencies

In [2]:
!pip install cmapy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for cmapy: filename=cmapy-0.6.6-py3-none-any.whl size=3959 sha256=d202485f0f2a3ba2dee7817dcd0f04de3d34fbfb46c71d6c98ddbfb26b363fc9
  Stored in directory: /root/.cache/pip/wheels/49/9a/c8/486d47fd07a9ca1b91602dd040f727313dbeaa059a45d77521
Successfully built cmapy


In [3]:
import os
import cv2
import random
import cmapy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import librosa as lb
import librosa.display as lbd
import soundfile as sf

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Load Data

In [14]:
root = '/content/drive/MyDrive/hack/datasets/'

In [15]:
df = pd.read_csv(root + 'processed.csv',index_col=0)
df.head()

,start,end,pid,mode,filename,disease,crack_wheeze
0,1.862,5.718,160,mc,160_1b3_Ar_mc_AKGC417L_0.wav,COPD,2
1,5.718,9.725,160,mc,160_1b3_Ar_mc_AKGC417L_1.wav,COPD,2
2,9.725,13.614,160,mc,160_1b3_Ar_mc_AKGC417L_2.wav,COPD,2
3,13.614,17.671,160,mc,160_1b3_Ar_mc_AKGC417L_3.wav,COPD,2
4,17.671,19.541,160,mc,160_1b3_Ar_mc_AKGC417L_4.wav,COPD,0


기존 original data 와 concat data(concatenated based augmentation data)가 합쳐진 dataframe 에서<br/> 
concat data의 dataframe 만 별도로 저장

In [ ]:
concat_df = pd.read_csv(root + 'concat_df.csv', index_col=[0])
concat_df.head()

,start,end,pid,mode,filename,disease,crack_wheeze
6898,NaN,NaN,NaN,NaN,aug_crackle_450.wav,NaN,1
6899,NaN,NaN,NaN,NaN,aug_crackle_451.wav,NaN,1
6900,NaN,NaN,NaN,NaN,aug_crackle_452.wav,NaN,1
6901,NaN,NaN,NaN,NaN,aug_crackle_453.wav,NaN,1
6902,NaN,NaN,NaN,NaN,aug_crackle_454.wav,NaN,1


In [ ]:
concat_df.reset_index(drop=True, inplace=True)
concat_df.head()

,start,end,pid,mode,filename,disease,crack_wheeze
0,NaN,NaN,NaN,NaN,aug_crackle_450.wav,NaN,1
1,NaN,NaN,NaN,NaN,aug_crackle_451.wav,NaN,1
2,NaN,NaN,NaN,NaN,aug_crackle_452.wav,NaN,1
3,NaN,NaN,NaN,NaN,aug_crackle_453.wav,NaN,1
4,NaN,NaN,NaN,NaN,aug_crackle_454.wav,NaN,1


In [ ]:
concat_df.drop(996, inplace=True)
concat_df.head()

,start,end,pid,mode,filename,disease,crack_wheeze
0,NaN,NaN,NaN,NaN,aug_crackle_450.wav,NaN,1
1,NaN,NaN,NaN,NaN,aug_crackle_451.wav,NaN,1
2,NaN,NaN,NaN,NaN,aug_crackle_452.wav,NaN,1
3,NaN,NaN,NaN,NaN,aug_crackle_453.wav,NaN,1
4,NaN,NaN,NaN,NaN,aug_crackle_454.wav,NaN,1


In [ ]:
concat_df.reset_index(drop=True, inplace=True)
concat_df.head()

,start,end,pid,mode,filename,disease,crack_wheeze
0,NaN,NaN,NaN,NaN,aug_crackle_450.wav,NaN,1
1,NaN,NaN,NaN,NaN,aug_crackle_451.wav,NaN,1
2,NaN,NaN,NaN,NaN,aug_crackle_452.wav,NaN,1
3,NaN,NaN,NaN,NaN,aug_crackle_453.wav,NaN,1
4,NaN,NaN,NaN,NaN,aug_crackle_454.wav,NaN,1


In [ ]:
concat_df[6130:]

,start,end,pid,mode,filename,disease,crack_wheeze
6130,NaN,NaN,NaN,NaN,aug_both_505.wav,NaN,3
6131,NaN,NaN,NaN,NaN,aug_both_506.wav,NaN,3
6132,NaN,NaN,NaN,NaN,aug_both_507.wav,NaN,3


In [ ]:
concat_df.to_csv(root + 'final_concat_df.csv')

## Padding
original data는 smart padding 진행 <br/>
concat data는 duplicated padding 진행

In [5]:
sr = 16000
maxLen = 7
org_root = root + 'processed_audio_files_no_pad/'

In [6]:
#slicing
def audio_slicing(signal, max = 7 * sr):
    if len(signal) > max:
        return signal[:max]
    else:
        return signal

In [7]:
#duplicated check length
def duplicated_check_length(audio, maxLen=maxLen):
    if len(audio) / sr >= maxLen:
        return audio_slicing(audio)

    else:
        new = np.concatenate((audio, audio))
        return duplicated_check_length(new)

In [8]:
def duplicated_padding(idx):
    audio_data, _ = lb.load(org_root + df.loc[idx, 'filename'], sr=sr)
    return (duplicated_check_length(np.concatenate((audio_data, audio_data))))

In [9]:
#smart padding check length
def smart_check_length(new_audio, idx_j, audio_i, audio_j):
    if len(new_audio)/sr >= maxLen:
        return audio_slicing(new_audio)
    else:
        if df['pid'][idx_j] == df['pid'][idx_j+1] and (df.loc[idx_j, 'crack_wheeze'] == df.loc[idx_j+1, 'crack_wheeze'] or df.loc[idx_j+1, 'crack_wheeze'] == 0):
            audio_k, _ = lb.load(org_root + df.loc[idx_j+1, 'filename'], sr=sr)
            new_audio_1 = np.concatenate((new_audio, audio_k))
            idx_j = idx_j + 1
            return smart_check_length(new_audio_1, idx_j, audio_i, audio_j)
        else: 
            prob = np.random.randint(0,2)
            if prob == 0:
                audio_1 = np.concatenate((new_audio, audio_i))
                return smart_check_length(audio_1, idx_j, audio_i, audio_j)
            else:
                audio_2 = np.concatenate((new_audio, audio_j))
                return smart_check_length(audio_2, idx_j, audio_i, audio_j)

In [10]:
def smart_padding(idx_i, idx_j):
    audio_i, _ = lb.load(org_root + df.loc[idx_i, 'filename'], sr=sr)
    audio_j, _ = lb.load(org_root + df.loc[idx_j, 'filename'], sr=sr)
    new_audio = np.concatenate((audio_i, audio_j))
    return smart_check_length(new_audio, idx_j, audio_i, audio_j)

#### smart padding for original data

In [ ]:
smart_pad=[]
for idx in range(len(df)):
  if idx == 6894:
    break

  if df['pid'][idx] == df['pid'][idx+1] and (df.loc[idx, 'crack_wheeze'] == df.loc[idx+1, 'crack_wheeze'] or df.loc[idx+1, 'crack_wheeze'] ==0):
      smart_pad.append(smart_padding(idx, idx+1))

  else: 
      smart_pad.append(duplicated_padding(idx))

In [ ]:
len(smart_pad)

In [ ]:
len(smart_pad[6893] / sr)

112000

6893번까지 smart padding 진행 <br/>
6894번부터 6897번까지는 duplicated padding 진행

In [ ]:
# 6894 ~ 6897 duplicated padding
for idx in range(6894, 6898):
    smart_pad.append(duplicated_padding(idx))

In [ ]:
len(smart_pad[6897] / sr)

112000

In [ ]:
np.save(root+'smart_pad.npy', smart_pad)

#### duplicated padding for concat data

In [ ]:
import tensorflow as tf
concat_audio = []
root_both = '/content/drive/MyDrive/hack/datasets/aug_both/'
root_crackle = '/content/drive/MyDrive/hack/datasets/aug_crackle/'
root_wheeze = '/content/drive/MyDrive/hack/datasets/aug_wheezing/'

a = tf.io.gfile.glob('/content/drive/MyDrive/hack/datasets/aug_both/*')
concat_audio = a + concat_audio
b = tf.io.gfile.glob('/content/drive/MyDrive/hack/datasets/aug_crackle/*')
concat_audio = b + concat_audio
c = tf.io.gfile.glob('/content/drive/MyDrive/hack/datasets/aug_wheezing/*')
concat_audio = c + concat_audio

In [ ]:
del concat_audio[999]

In [ ]:
len(concat_audio)

6134

In [ ]:
#duplicated check length
def check_length(audio, maxLen=maxLen):
    if len(audio) / sr >= maxLen:
        return audio_slicing(audio)

    else:
        new = np.concatenate((audio, audio))
        return check_length(new)

In [ ]:
s, _ = lb.load(concat_audio[3200], sr=sr)

In [ ]:
concat_audio[3200]

'/content/drive/MyDrive/hack/datasets/aug_crackle/aug_crackle_27.wav'

In [ ]:
print(concat_df[concat_df['filename']=='aug_crackle_27.wav'])

     start  end  pid  mode            filename  disease  crack_wheeze
996    NaN  NaN  NaN   NaN  aug_crackle_27.wav      NaN             1


In [ ]:
concat_df['filename'][996]

'aug_crackle_27.wav'

In [ ]:
s

array([], dtype=float32)

In [ ]:
del concat_audio[3200]

In [ ]:
len(concat_audio)

6133

In [ ]:
len(concat_df)

6133

In [ ]:
concat_7sec = []
concat_7sec_label = []

for i, (idx, row) in enumerate(concat_df.iterrows()):
    filename = row['filename']
    label = row['crack_wheeze']
    audio_data, _ = lb.load(concat_audio[i], sr=sr)
    concat_7sec_audio = check_length(audio_data)
    concat_7sec.append(concat_7sec_audio)
    concat_7sec_label.append(label)
    save_path = root + 'concat_7sec/' + filename

    sf.write(file=save_path, data=concat_7sec_audio, samplerate=sr)

In [ ]:
np.save(root+'concat_dupli_pad.npy', concat_7sec)
np.save(root+'concat_dupli_pad_label.npy', concat_7sec_label)

concatenated based augmentation dataset 을 7초에 맞게 duplicated padding<br/>
wav 파일로 저장<br/>
concat_7sec = signal data <br/>
concat_7sec_label = signal data label